# Microsoft Azure AutoML Demo

## Purpose and Challenge

The purpose of this notebook is for the user to build and deploy a Machine Learning (ML) application using Azure Machine Learning (AML) Service which is in preview, end-2-end. The challenge we will tackle is a simple predictive maintenance solution.

This notebook has the complete code to load, prep, train and deploy the model. We chose a small public data set for this demo so as to run the entire process in only few minutes. Please note that AML Service is in preview and is being updated constantly to address any issues and add new functionality. 

Also, this notebook has been tested using Azure Notebooks Service (preview) so as to do the demo all on Azure!!!

Following are the high level steps:

1. Acquire and Prepare Data
2. Automated ML
3. Deploy Model



## 1. Acquire and Prepare Data
For this notebook, we will use the NASA Prognostics Center's Turbo-Fan Failure dataset.  It is located here: https://ti.arc.nasa.gov/tech/dash/groups/pcoe/prognostic-data-repository/#turbofan

We have it as .txt file in the same folder. We read it into a Pandas DataFrame.
Note the headers were not in the space seperated txt file, so we assign them from the ReadMe in the zip file. In pandas we use read_csv with the delimiter option, even with a space delimited file.

In [91]:
import pandas as pd
train = pd.read_csv("train_FD001.txt", delimiter="\s|\s\s", index_col=False, engine='python', names=['unit','cycle','os1','os2','os3','sm1','sm2','sm3','sm4','sm5','sm6','sm7','sm8','sm9','sm10','sm11','sm12','sm13','sm14','sm15','sm16','sm17','sm18','sm19','sm20','sm21'])

Take a quick look at the data

In [92]:
train.head(5)

,unit,cycle,os1,os2,os3,sm1,sm2,sm3,sm4,sm5,...,sm12,sm13,sm14,sm15,sm16,sm17,sm18,sm19,sm20,sm21
0,1,1,-0.0007,-0.0004,100.0,518.67,641.82,1589.70,1400.60,14.62,...,521.66,2388.02,8138.62,8.4195,0.03,392,2388,100.0,39.06,23.4190
1,1,2,0.0019,-0.0003,100.0,518.67,642.15,1591.82,1403.14,14.62,...,522.28,2388.07,8131.49,8.4318,0.03,392,2388,100.0,39.00,23.4236
2,1,3,-0.0043,0.0003,100.0,518.67,642.35,1587.99,1404.20,14.62,...,522.42,2388.03,8133.23,8.4178,0.03,390,2388,100.0,38.95,23.3442
3,1,4,0.0007,0.0000,100.0,518.67,642.35,1582.79,1401.87,14.62,...,522.86,2388.08,8133.83,8.3682,0.03,392,2388,100.0,38.88,23.3739
4,1,5,-0.0019,-0.0002,100.0,518.67,642.37,1582.85,1406.22,14.62,...,522.19,2388.04,8133.80,8.4294,0.03,393,2388,100.0,38.90,23.4044


Our dataset has a number of units in it, with each engine flight listed as a cycle. The cycles count up until the engine fails. What we would like to predict is the no. of cycles until failure. 
So we need to calculate a new column called RUL, or Remaining Useful Life.  It will be the last cycle value minus each cycle value per unit.

In [93]:
# Assign ground truth
def assignrul(df):
    maxi = df['cycle'].max()
    df['rul'] = maxi - df['cycle']
    return df
    

train_new = train.groupby('unit').apply(assignrul)

train_new.columns

Index(['unit', 'cycle', 'os1', 'os2', 'os3', 'sm1', 'sm2', 'sm3', 'sm4', 'sm5',
       'sm6', 'sm7', 'sm8', 'sm9', 'sm10', 'sm11', 'sm12', 'sm13', 'sm14',
       'sm15', 'sm16', 'sm17', 'sm18', 'sm19', 'sm20', 'sm21', 'rul'],
      dtype='object')

Now our dataframe has the 'RUL' column.  Predicting this value will be the objective of this exercise.

In [94]:
train_new.head(5)

,unit,cycle,os1,os2,os3,sm1,sm2,sm3,sm4,sm5,...,sm13,sm14,sm15,sm16,sm17,sm18,sm19,sm20,sm21,rul
0,1,1,-0.0007,-0.0004,100.0,518.67,641.82,1589.70,1400.60,14.62,...,2388.02,8138.62,8.4195,0.03,392,2388,100.0,39.06,23.4190,191
1,1,2,0.0019,-0.0003,100.0,518.67,642.15,1591.82,1403.14,14.62,...,2388.07,8131.49,8.4318,0.03,392,2388,100.0,39.00,23.4236,190
2,1,3,-0.0043,0.0003,100.0,518.67,642.35,1587.99,1404.20,14.62,...,2388.03,8133.23,8.4178,0.03,390,2388,100.0,38.95,23.3442,189
3,1,4,0.0007,0.0000,100.0,518.67,642.35,1582.79,1401.87,14.62,...,2388.08,8133.83,8.3682,0.03,392,2388,100.0,38.88,23.3739,188
4,1,5,-0.0019,-0.0002,100.0,518.67,642.37,1582.85,1406.22,14.62,...,2388.04,8133.80,8.4294,0.03,393,2388,100.0,38.90,23.4044,187


First note that the sensor measurements do seem to be changing as we near 0 RUL. This implies that we should be able to make a model that will be useful enough for business value.

We are now ready to train a model on this data using Automated ML.

## 3. automated ML

Here we utilize Azure's AutoML package to automate the scaling of the sensors, selection of sensors, and automatically train and evaluate many different types of ML models.

Import Azure ML libs for automated ML

In [95]:
import logging
import os
import random
import time

from matplotlib import pyplot as plt
from matplotlib.pyplot import imshow
import numpy as np
import pandas as pd

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.train.automl.run import AutoMLRun
from azureml.widgets import RunDetails
from azureml.core.model import Model

Provide your Machine Learning Workspace credentials to run AutoML. You will need to perform Microsoft's MFA. Please follow the manual auth instructions.

In [96]:
subscription_id = "<Your SubscriptionId>" #you should be owner or contributor
resource_group = "<Resource group - new or existing>" #you should be owner or contributor
workspace_name = "<workspace to be created>" #your workspace name
workspace_region = "<azureregion>" #your region

#Example here, use your own
subscription_id = "381b38e9-9840-4719-a5a0-61d9585e1e91" #you should be owner or contributor
resource_group = "automl_nasa_newrg" #you should be owner or contributor
workspace_name = "automatedml_nasa_aznb" #your workspace name
workspace_region = "eastus2" #your region

Create Azure ML workspace

In [97]:
# Import the Workspace class and check the Azure ML SDK version.
from azureml.core import Workspace

ws = Workspace.create(name = workspace_name,
                      subscription_id = subscription_id,
                      resource_group = resource_group, 
                      location = workspace_region,                      
                      exist_ok=True)
ws.get_details()

{'id': '/subscriptions/381b38e9-9840-4719-a5a0-61d9585e1e91/resourceGroups/automl_nasa_newrg/providers/Microsoft.MachineLearningServices/workspaces/automatedml_nasa_aznb',
 'name': 'automatedml_nasa_aznb',
 'location': 'eastus2',
 'type': 'Microsoft.MachineLearningServices/workspaces',
 'workspaceid': '38a769cf-f8b3-424c-a906-838d79ac1ad4',
 'description': '',
 'friendlyName': 'automatedml_nasa_aznb',
 'creationTime': '2019-02-17T05:34:39.4332596+00:00',
 'containerRegistry': '/subscriptions/381b38e9-9840-4719-a5a0-61d9585e1e91/resourcegroups/automl_nasa_newrg/providers/microsoft.containerregistry/registries/automateacrpezaacbj',
 'keyVault': '/subscriptions/381b38e9-9840-4719-a5a0-61d9585e1e91/resourcegroups/automl_nasa_newrg/providers/microsoft.keyvault/vaults/automatekeyvaultkpirsgjr',
 'applicationInsights': '/subscriptions/381b38e9-9840-4719-a5a0-61d9585e1e91/resourcegroups/automl_nasa_newrg/providers/microsoft.insights/components/automateinsightsblxrzkqy',
 'identityPrincipalId':

Write config to cluster

In [98]:
from azureml.core import Workspace

ws = Workspace(workspace_name = workspace_name,
               subscription_id = subscription_id,
               resource_group = resource_group)

# Persist the subscription id, resource group name, and workspace name in aml_config/config.json.
ws.write_config()

Wrote the config file config.json to: /home/nbuser/library/aml_config/config.json


Define the experiment name

In [99]:
# Choose a name for the experiment and specify the project folder.
experiment_name = 'automl-predictive-rul'
project_folder = './sample_projects/automl-demo-predmain'

experiment = Experiment(ws, experiment_name)

output = {}
output['SDK version'] = azureml.core.VERSION
output['Subscription ID'] = ws.subscription_id
output['Workspace Name'] = ws.name
output['Resource Group'] = ws.resource_group
output['Location'] = ws.location
output['Project Directory'] = project_folder
output['Experiment Name'] = experiment.name
pd.set_option('display.max_colwidth', -1)
pd.DataFrame(data = output, index = ['']).T

,
Experiment Name,automl-predictive-rul
Location,eastus2
Project Directory,./sample_projects/automl-demo-predmain
Resource Group,automl_nasa_newrg
SDK version,1.0.10
Subscription ID,381b38e9-9840-4719-a5a0-61d9585e1e91
Workspace Name,automatedml_nasa_aznb


Create training data

In [100]:
# put training data into X and Y arrays
X_train = train_new.iloc[:,2:26].values
y_train = train_new.iloc[:,26:27].values.astype(int).flatten()

Format data into arrays that AutoML will use to train models.  Below is one row of X and Y as an example.

In [101]:
X_train[0]

array([-7.00000e-04, -4.00000e-04,  1.00000e+02,  5.18670e+02,
        6.41820e+02,  1.58970e+03,  1.40060e+03,  1.46200e+01,
        2.16100e+01,  5.54360e+02,  2.38806e+03,  9.04619e+03,
        1.30000e+00,  4.74700e+01,  5.21660e+02,  2.38802e+03,
        8.13862e+03,  8.41950e+00,  3.00000e-02,  3.92000e+02,
        2.38800e+03,  1.00000e+02,  3.90600e+01,  2.34190e+01])

In [102]:
y_train[0]

191

Enable Telemetry

In [103]:
from azureml.telemetry import set_diagnostics_collection
set_diagnostics_collection(send_diagnostics = True)

Turning diagnostics collection on. 


Now we are ready to configure automated ML.  We provide necessary information on: what we want to predict, what accuracy metric we want to use, how many models we want to try, and many other parameters.  AutoML will also automatically scale the data for us.

## Configure Automated ML

You can use these params.

|Property|Description|
|-|-|
|**task**|classification or regression|
|**primary_metric**|This is the metric that you want to optimize. Classification supports the following primary metrics: <br><i>accuracy</i><br><i>AUC_weighted</i><br><i>average_precision_score_weighted</i><br><i>norm_macro_recall</i><br><i>precision_score_weighted</i>|
|**primary_metric**|This is the metric that you want to optimize. Regression supports the following primary metrics: <br><i>spearman_correlation</i><br><i>normalized_root_mean_squared_error</i><br><i>r2_score</i><br><i>normalized_mean_absolute_error</i>|
|**iteration_timeout_minutes**|Time limit in minutes for each iteration.|
|**iterations**|Number of iterations. In each iteration AutoML trains a specific pipeline with the data.|
|**n_cross_validations**|Number of cross validation splits.|
|**X**|(sparse) array-like, shape = [n_samples, n_features]|
|**y**|(sparse) array-like, shape = [n_samples, ], [n_samples, n_classes]<br>Multi-class targets. An indicator matrix turns on multilabel classification. This should be an array of integers.|
|**path**|Relative path to the project folder. AutoML stores configuration files for the experiment under this folder. You can specify a new empty folder.|
|**preprocess**|set this to True to enable pre-processing of data eg. string to numeric using one-hot encoding|
|**exit_score**|Target score for experiment. It is associated with the metric. eg. exit_score=0.995 will exit experiment after that|

In [123]:
##Local compute 
Automl_config = AutoMLConfig(task = 'regression',
                             primary_metric = 'r2_score',
                             iteration_timeout_minutes = 5,
                             iterations = 10,
                             n_cross_validations = 3,
                             max_cores_per_iteration = 1,
                             preprocess = False,
                             experiment_exit_score = 0.985,
                             #blacklist_models = ['kNN','LinearSVM', 'ExtremeRandomTrees', 'RandomForestRegressor'],
                             whitelist_models = ['ExtremeRandomTrees','ElasticNet','LightGBM'],
                             X = X_train,
                             y = y_train,
                             debug_log = 'automl_errors.log',
                             verbosity=logging.ERROR,
                             path=project_folder)

Finally we are ready to launch AutoML.  This step can take many minutes, but AutoML will give you updates as models are trained and evaluated by the metric we specified above.  AutoML also let us know which scaling method was used.  The information from each ML model training will be stored in the Experiment section of the ML Workspace, where we can review it through Azure Portal.

In [124]:
# Train multiple models using AutoML
experiment=Experiment(ws, experiment_name)
local_run = experiment.submit(Automl_config, show_output=True)

Running on local machine
Parent Run ID: AutoML_7209ff4c-5775-40bc-90f0-c02b4440ea1c
*******************************************************************************************************************
ITERATION: The iteration being evaluated.
PIPELINE: A summary description of the pipeline being evaluated.
TRAINFRAC: Fraction of the training data to train on.
DURATION: Time taken for the current iteration.
METRIC: The result of computing score on the fitted pipeline.
BEST: The best observed score thus far.
*******************************************************************************************************************

 ITERATION   PIPELINE                                       TRAINFRAC  DURATION      METRIC      BEST
         0   SparseNormalizer ExtremeRandomTrees            1.0000     0:02:58       0.6355    0.6355
         1   StandardScalerWrapper LightGBM                 1.0000     0:02:19       0.6252    0.6355
         2   StandardScalerWrapper ExtremeRandomTrees       1.0000

Widget UX

In [125]:
RunDetails(local_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': True, 'log_level': 'INFO', 'sd…

We now get the best model

In [126]:
# find the run with the highest accuracy value.
best_run, fitted_model = local_run.get_output()
print(best_run)

Run(Experiment: automl-predictive-rul,
Id: AutoML_7209ff4c-5775-40bc-90f0-c02b4440ea1c_0,
Type: None,
Status: Completed)


## 3. Deploy Model

In [127]:
# register best model in workspace
description = 'AutoML-RUL-Regression-20190213'
tags = None
local_run.register_model(description=description, tags=tags)
local_run.model_id # Use this id to deploy the model as a web service in Azure

Registering model AutoML7209ff4c5best


'AutoML7209ff4c5best'

After we register the model in our ML Workspace, it should be visible in Azure Portal.

Now we want to deploy the model as a REST API that we can feed a row or rows of "X" data to, and return the predicted 'RUL' value.  To accomplish this, we will build a container image in our AML Workspace and deploy that image as a Container instance in Azure's ACI service.  We will then obtain an IP address where we can submit data and receive back the predicted 'RUL' value.

There are 3 things we need: 
1. A score.py file that contains the init() and run() functions with instructions on how to load and socre with the model
2. A myenv.yml file that contains information on the python environment in which the model needs to run
3. Configurations for our images and our services, using functions provided by AzureML service.

The cells below help you set these up.   You will need to use the registered model name provided by the cell above.

In [128]:
%%writefile score.py
# Scoring Script
import json
import numpy as np
import os
import pickle
from sklearn.externals import joblib
from sklearn.linear_model import LogisticRegression

from azureml.core.model import Model

import azureml.train.automl

def init():
    global model
    # retreive the path to the model file using the model name
    model_path = Model.get_model_path('AutoML7209ff4c5best')
    print(model_path)
    model = joblib.load(model_path)
    

def run(raw_data):
    # grab and prepare the data
    data = (np.array(json.loads(raw_data)['data'])).reshape(1,-1)
    # make prediction
    y_hat = model.predict(data)
    return json.dumps(y_hat.tolist())

Overwriting score.py


In [129]:
from azureml.core.conda_dependencies import CondaDependencies

myenv = CondaDependencies.create(conda_packages=['numpy','scikit-learn','lightgbm'], pip_packages=['azureml-sdk[automl]'])

conda_env_file_name = 'myenv.yml'
myenv.save_to_file('.', conda_env_file_name)

'myenv.yml'

In [130]:
from azureml.core.conda_dependencies import CondaDependencies 

myenv = CondaDependencies()
myenv.add_conda_package("scikit-learn")

with open("myenv.yml","w") as f:
    f.write(myenv.serialize_to_string())

In [131]:
with open("myenv.yml","r") as f:
    print(f.read())

# Conda environment specification. The dependencies defined in this file will
# be automatically provisioned for runs with userManagedDependencies=False.

# Details about the Conda environment file format:
# https://conda.io/docs/user-guide/tasks/manage-environments.html#create-env-file-manually

name: project_environment
dependencies:
  # The python interpreter version.
  # Currently Azure ML only supports 3.5.2 and later.
- python=3.6.2

- pip:
    # Required packages for AzureML execution, history, and data preparation.
  - azureml-defaults
- scikit-learn



*If the above file does not show "- azureml-train-automl" in the pip section, the you need to add it manually*

Now configure the webservice.

In [132]:
from azureml.core.webservice import AciWebservice

aciconfig = AciWebservice.deploy_configuration(cpu_cores=2, 
                                               memory_gb=2, 
                                               tags={"data": "RUL",  "method" : "sklearn"}, 
                                               description='Predict RUL with Azure AutoML')

Finally, configure the container image and deploy the service. Make sure the filenames match, your Workspace is in variable ws, and your model name is correct. It will create your containter image and deploy it as a webservice.

This process can take up to 10 minutes, so please be patient. You can check the progress bar periodically ...

In [ ]:
%%time
from azureml.core.webservice import Webservice
from azureml.core.image import ContainerImage

# configure the image
image_config = ContainerImage.image_configuration(execution_script="score.py", 
                                                  runtime="python", 
                                                  conda_file="myenv.yml")

service = Webservice.deploy_from_model(workspace=ws,
                                       name='automl-rul-regress5',
                                       deployment_config=aciconfig,
                                       models=['AutoML7209ff4c5best:1'],
                                       image_config=image_config)

service.wait_for_deployment(show_output=True)

Creating image
Image creation operation finished for image automl-rul-regress5:1, operation "Succeeded"
Creating service
Running...............

Just as a check, we can retrieve the URI for the scoring function.

In [ ]:
print(service.scoring_uri)

Let's check to see if the service is working.  Here we submit a single row of data from X_train to see if it returns a reasonable prediction.

In [ ]:
import requests
import json

# send a random row from the test set to score
#random_index = np.random.randint(0, len(X_train)-1)
input_data = "{\"data\": " + str(list(X_train[0].reshape(1,-1)[0])) + "}"

headers = {'Content-Type':'application/json'}

# for AKS deployment you'd need to the service key in the header as well
# api_key = service.get_key()
# headers = {'Content-Type':'application/json',  'Authorization':('Bearer '+ api_key)} 

resp = requests.post(service.scoring_uri, input_data, headers=headers)

print("POST to url", service.scoring_uri)
print("input data:", input_data)
print("label:", y_train[0:1])
print("prediction:", resp.text)

Here we see one engine evolving through many flights, or cycles.  As we approach failure, the rul declines to zero, as does the prediction.  This is a good example of how the predictive model can assist in estimate the future failure of the engine.

Note that the model does not perform well at high rul.  This is an acceptable outcome as the engine is far from failure.

To avoid any run-away Azure costs, we always delete un-necessary services when we are done.

In [117]:
service.delete()